# Recursive data types and functions

## Recursive types

### The `List` type

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the algebraic following equation:

`IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



The implementation in Scala is as follows (we already give the generic version `List[A]`, rather than the implementation of `IntList`):

Note that the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. However, this forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times. The standard definition looks as follows:

In [ ]:
object StdDefinition{
}

We will stick to the former definition. Some examples of lists: 

In [ ]:
// The empty list


In [ ]:
// Non-empty list [1, 2, 3]


### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [ ]:
import scala.{List => IList}



How can we do that with out own lists? We define a smart constructor in the companion object using variadic arguments: 

In [ ]:
object List{
}

This allows us to write lists more easily:

Note that the smart constructor `apply` is defined recursively. Let's dive into recursion.

##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [ ]:
// Using mutable variables



The recursive function is implemented as follows: 

In [ ]:
// Using recursive functions



Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that we have available so far, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with lists of enough lenght.

### Tail-recursive functions

The implementation using tail-recursion solves the problems with the stack. It commonly makes use of auxiliary functions:

In [ ]:
// Using tail-recursive functions



We can check the stack-safety problems of non-tail recursive functions, by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [ ]:
// First, imperatively



In [ ]:
// Next, tail-recursively



Now, let's calculate the length of a list long enough, using each of the three implementations:

In [ ]:
// Imperatively


In [ ]:
// Tail-recursive


In [ ]:
// Plain recursive


### Using the standard `List` type

In [ ]:
import scala.collection.immutable.List

From now on, we will use the `List` type defined in the standard library of Scala. For the sake of comparison, let's re-implement the `length` function: 

In [ ]:
/*
def length[A](list: List[A]): Int = 
    list match {
        case Empty()           => 0
        case NonEmpty(_, tail) => 1 + length(tail)
    }
*/


### Unit testing with `scalatest`

In [16]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. For instance, this is a possible test class for the `length` function:

In [ ]:
/*
assert(length(List()) == 0)
assert(length(List(1)) == 1)
assert(length(List(1,2,3,4)) == 4)
*/

The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

### Example: adding numbers

Let's implement a function that sums all the numbers of a list.

In [ ]:
class TestSum extends FlatSpec with Matchers{
    "length" should "work" in {
    }
}

In [ ]:
// Recursively



In [ ]:
// With tail-recursion



### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [2]:
class TestProduct(product: List[Int] => Int) 
extends FlatSpec with Matchers{
    "product" should "work" in {
        product(List(1,2,3)) shouldBe 6
        product(List(1,4,2,5)) shouldBe 40
        product(List()) shouldBe 1
        product(List(0,1,2,3,4,5,6)) shouldBe 0
    }
}

defined class TestProduct

This is the common recursive implementation:

In [4]:
def product(list: List[Int]): Int = 
    list match {
        case Nil => 1 : Int
        case head :: tail => head * product(tail) : Int 
    }

defined function product

It works as expected: 

In [5]:
run(new TestProduct(product))

cmd1$Helper$TestProduct:
product
- should work


But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [8]:
def product(list: List[Int]): Int =
    list match {
        case Nil => 1
        case head :: tail => 
            if (head == 0) 0
            else head * product(tail)
    }


defined function product

In [9]:
def product(list: List[Int]): Int =
    list match {
        case Nil => 1
        case head :: tail if head == 0 => 0
        case head :: tail => head * product(tail)
    }


defined function product

In [11]:
def product(list: List[Int]): Int =
    list match {
        case Nil => 1
        case 0 :: _ => 0
        case head :: tail => head * product(tail)
    }


defined function product

In [12]:
run(new TestProduct(product))

cmd1$Helper$TestProduct:
product
- should work


A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [17]:
//class TestMember(member: Function2[List[Int], Int, Boolean]) 
class TestMember(member: (List[Int], Int) => Boolean) 
extends FlatSpec with Matchers{
    "member" should "work" in {
        member(List(1,2,3), 3) shouldBe true
        member(List(1,2,3), 4) shouldBe false
        member(List(), 6) shouldBe false
    }
}

defined class TestMember

In [18]:
def member(l: List[Int], e: Int): Boolean = 
    ??? : Boolean 

defined function member

In [18]:
def member(l: List[Int], e: Int): Boolean = 
    l match {
        case Nil => ??? : Boolean 
        case head :: tail => ??? : Boolean 
    }

defined function member

In [19]:
def member(l: List[Int], e: Int): Boolean = 
    l match {
        case Nil => 
            false : Boolean 
        case head :: tail => ??? : Boolean 
    }

defined function member

In [20]:
def member(l: List[Int], e: Int): Boolean = 
    l match {
        case Nil => 
            false : Boolean 
        case head :: tail => 
            val memberTail: Boolean = member(tail, e)
            ??? : Boolean 
            //if (head == e) true
            //else member(tail) : Boolean 
    }

defined function member

In [21]:
def member(l: List[Int], e: Int): Boolean = 
    l match {
        case Nil => 
            false : Boolean 
        case head :: tail => 
            val memberTail: Boolean = member(tail, e)
            (head == e || memberTail) : Boolean 
            //if (head == e) true
            //else member(tail) : Boolean 
    }

defined function member

In [21]:
def member(l: List[Int], e: Int): Boolean = 
    l match {
        case Nil => 
            false : Boolean 
        case head :: tail => 
            (head == e || memberTail(tail, e)) : Boolean 
            //if (head == e) true
            //else member(tail) : Boolean 
    }

defined function member

In [40]:
def member[A](l: List[A], e: A): Boolean = 
    l match {
        case Nil => 
            false : Boolean 
        case `e` :: _ => true
        case _ :: tail => member(tail, e)
    }

defined function member

In [31]:
val member: (List[Int], Int) => Boolean = 
    (l: List[Int], i: Int) => ??? : Boolean 

member: (List[Int], Int) => Boolean = ammonite.$sess.cmd30$Helper$$Lambda$3005/1639363629@72c860ca

In [34]:
val memberV: (List[Int], Int) => Boolean = 
    (l: List[Int], i: Int) => 
        l match {
            case Nil => false 
            case head :: tail => head == i || memberV(tail, i)
        }

memberV: (List[Int], Int) => Boolean = ammonite.$sess.cmd33$Helper$$Lambda$3032/1178768937@7bda8264

In [42]:
// val memberV: [A] ==> (List[A], A) => Boolean 
def memberV[A]: (List[A], A) => Boolean = 
    {
        case (Nil, _) => false 
        case (head :: tail, i) => head == i || memberV(tail, i)
    }

defined function memberV

In [44]:
run(new TestMember(memberV))

cmd16$Helper$TestMember:
member
- should work


We can also pattern match against a specific value as follows:

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [46]:
class TestLast(last: List[Int] => Either[Int, Unit]) extends FlatSpec with Matchers{
    "last" should "work" in {
        last(List(1,2,3)) shouldBe Left(3: Int)
        last(List(1)) shouldBe Left(1: Int)
        last(List()) shouldBe Right(() : Unit)
    }
}

defined class TestLast

In [47]:
class TestLast(last: List[Int] => Option[Int]) extends FlatSpec with Matchers{
    "last" should "work" in {
        last(List(1,2,3)) shouldBe Some(3: Int)
        last(List(1)) shouldBe Some(1: Int)
        last(List()) shouldBe None
    }
}

defined class TestLast

In [ ]:
def last[A](l: List[A]): Option[A] = 
    ??? : Option[A]

In [48]:
def last[A](l: List[A]): Option[A] = 
    l match {
        case Nil => ??? : Option[A]
        case head :: tail => ??? : Option[A]
    }

defined function last

In [49]:
def last[A](l: List[A]): Option[A] = 
    l match {
        case Nil => 
            // Some(??? : A) : Option[A]
            None : Option[A]
        case head :: tail => ??? : Option[A]
    }

defined function last

In [52]:
def last[A](l: List[A]): Option[A] = 
    l match {
        case Nil => 
            // Some(??? : A) : Option[A]
            None : Option[A]
        case head :: tail => 
            if (tail == Nil) Some(head)
            else last(tail) // : Option[A]
    }

defined function last

In [58]:
def last[A](l: List[A]): Option[A] = 
    l match {
        case Nil => 
            // Some(??? : A) : Option[A]
            None : Option[A]
        case head :: Nil => 
            Some(head)
        //case _ :: (tail@(h2 :: tail2)) => 
        case _ :: tail => 
            last(tail) // : Option[A]
    }

defined function last

In [59]:
run(new TestLast(last))

cmd46$Helper$TestLast:
last
- should work


### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [66]:
class TestInsertLast(insert: (List[Int], Int) => List[Int]) 
extends FlatSpec with Matchers{
    "insertLast" should "work for the empty list" in {
        insert(List(), 3) shouldBe List(3)
    }
    it should "work for non-empty list" in {
        insert(List(1,2,2), 3) shouldBe List(1,2,2,3)
        insert(1 :: List(2,2), 3) shouldBe 1 :: List(2,2,3)
        insert(List(1), 5) shouldBe List(1,5)
    }
}

defined class TestInsertLast

In [ ]:
def insertLast[A](l: List[A], e: A): List[A] = 
    ??? : List[A]

In [61]:
def insertLast[A](l: List[A], e: A): List[A] = 
    l match {
        case Nil => ??? : List[A]
        case head :: tail => ??? : List[A]
    }

defined function insertLast

In [62]:
def insertLast[A](l: List[A], e: A): List[A] = 
    l match {
        case Nil => 
            List(e) : List[A]
        case head :: tail => 
            ??? : List[A]
    }

defined function insertLast

In [62]:
def insertLast[A](l: List[A], e: A): List[A] = 
    l match {
        case Nil => 
            List(e) : List[A]
        case head :: tail => 
            val tailSol: List[A] = insertLast(tail, e)
            ??? : List[A]
    }

defined function insertLast

In [67]:
def insertLast[A](l: List[A], e: A): List[A] = 
    l match {
        case Nil => 
            List(e) : List[A]
        case head :: tail => 
            val tailSol: List[A] = insertLast(tail, e)
            head :: tailSol : List[A]
    }

defined function insertLast

In [72]:
def insertLast[A](l: List[A], e: A): List[A] = 
    l match {
        case Nil => 
            List(e)
        case head :: tail => 
            head :: insertLast(tail, e)
    }

defined function insertLast

In [73]:
run(new TestInsertLast(insertLast))

cmd65$Helper$TestInsertLast:
insertLast
- should work for the empty list
- should work for non-empty list


### Example: concatenate lists

Let's implement this function step-by-step, following the types. We start from the signature of the desired function:

In [ ]:
object TestConcatenate extends FlatSpec with Matchers{
    "concatenate" should "work" in {
    }
}

In [ ]:
// Recursive

In [ ]:
run(TestConcatenate)

In [ ]:
// Tail-recursive

In [ ]:
run(TestConcatenate)

### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [74]:
class TestReverse(reverse: List[Int] => List[Int]) extends FlatSpec with Matchers{
    "reverse" should "work" in {
        reverse(List(1,2,3)) shouldBe List(3,2,1)
        reverse(List(52,26,2,2)) shouldBe List(2,2,26,52)
        reverse(52 :: List(26,2,2)) shouldBe List(2,2,26) // List(2,2,26,52)
        reverse(List()) shouldBe List()
        reverse(List(1)) shouldBe List(1)
        reverse(List(1,2,3,2,1)) shouldBe List(1,2,3,2,1)
    }
}

defined class TestReverse

In [76]:
// Recursively: Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    ??? : List[A]

defined function reverse

In [77]:
// Recursively: Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    l match {
        case Nil => ??? : List[A]
        case head :: tail => ??? : List[A]
    } 

defined function reverse

In [78]:
// Recursively: Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    l match {
        case Nil => 
            // List.apply() : List[A]
            Nil 
        case head :: tail => ??? : List[A]
    } 

defined function reverse

In [79]:
// Recursively: Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    l match {
        case Nil => 
            // List.apply() : List[A]
            Nil 
        case List(e) => 
            List(e)
        case head :: tail => 
            val tailSol: List[A] = reverse(tail) 
            ??? : List[A]    
    } 

defined function reverse

In [83]:
// Recursively: Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    l match {
        case Nil => 
            // List.apply() : List[A]
            Nil 
        case List(e) => 
            List(e)
        case head :: tail => 
            val tailSol: List[A] = reverse(tail) 
            tailSol :+ head : List[A]
            
    } 

defined function reverse

In [95]:
// Recursively: Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    l match {
        case Nil => 
            // List.apply() : List[A]
            Nil 
        case List(e) => 
            List(e)
        case head :: tail => 
            reverse(tail) :+ head
            
    } 

defined function reverse

In [98]:
// Recursively: Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    l match {
        case Nil => 
            // List.apply() : List[A]
            Nil 
        case head :: tail => 
            reverse(tail) :+ head
            
    } 

defined function reverse

In [84]:
List(1,2,3) :+ 5

res83: List[Int] = List(1, 2, 3, 5)

In [85]:
run(new TestReverse(reverse))

cmd73$Helper$TestReverse:
reverse
- should work


In [89]:
reverse(List.fill(500000)(2))

: 

In [91]:
reverse(List(1,2,3))
/*
out, aux
========
Nil, 1::2::3::Nil
1::Nil, 2::3::Nil
2::1::Nil, 3::Nil
3::2::1::Nil, Nil
*/
// Tail-recursive, efficiently
def reverseTR[A](l: List[A]): List[A] = {
    def auxReverse(out: List[A], aux: List[A]): List[A] = 
        aux match {
            case Nil => out
            case head :: tail => auxReverse(head :: out, tail)
        }
    auxReverse(Nil, l)
}


res90_0: List[Int] = List(3, 2, 1)
defined function reverseTR

In [92]:
run(new TestReverse(reverseTR))

cmd73$Helper$TestReverse:
reverse
- should work


In [97]:
reverseTR(List.fill(1000000)(0))

res96: List[Int] = List(
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
...